Imports

In [1]:
#!pip install gradio
import gradio
import requests
from bs4 import BeautifulSoup

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM


/home/bigdata/miniconda3/envs/ollama_environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Select model for embeddings

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Access existing database

In [3]:
vector_store = Chroma(
    collection_name = "info_about_Spain",
    embedding_function = embeddings,
    persist_directory = "./chroma_info_about_Spain",
)

Define retriever

In [4]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 5}
)

RAG chain

In [5]:
# Prompt
template = """Answer the question based mostly on the following context:
{context}
If you can't respond with the given context, avoid responding.

Question: {question}

Answer: always in english. Be concise.
"""
prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="tinyllama", base_url='http://localhost:11434')

# Define the chain
chain = prompt | model

Function to generate a response given a question

In [6]:
def ask_gui(query):
    response = chain.invoke({"context": retriever, "question": query})
    return response

Create the GUI

In [7]:
with gradio.Blocks() as demo:
    gradio.Markdown("## RAG SYSTEM")
    
    with gradio.Row():
        query_input = gradio.Textbox(label="Enter Your Query", placeholder="Type your question here...")

    search_button = gradio.Button("SEND")
    output_box = gradio.Textbox(label="Search Results", lines=15)

    # Bind the function to the gradio UI
    search_button.click(fn=ask_gui, inputs=[query_input], outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
